In [82]:
import sqlite3
import pandas as pd

CATALOG_DB_PATH = "/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db"
catalog_conn = sqlite3.connect(CATALOG_DB_PATH)
catalog_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize','app','app_id'])
catalog_df = pd.read_sql_query("SELECT block_hashes.obj_id, block_hashes.inode, block_hashes.filename, block_hashes.file_offset, \
                block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize , files.app, files.app_id\
                FROM files \
                INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id \
                and files.inode = block_hashes.inode and files.filename=block_hashes.filename;", catalog_conn)
print("original length ",len(catalog_df))
catalog_df = catalog_df[catalog_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
catalog_df = catalog_df[catalog_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
catalog_df = catalog_df[catalog_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
catalog_df = catalog_df[catalog_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
catalog_conn.close()
print("catalog app length, ", len(catalog_df))
#return appdf

original length  6826014
catalog app length,  6546710


In [83]:
IMAGE_DB_PATH = "/Users/seunfuta/Downloads/NIST/IMG/Wireshark-W7x32.db"
image_conn = sqlite3.connect(IMAGE_DB_PATH)
image_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize'])
image_df = pd.read_sql_query("SELECT block_hashes.obj_id, files.inode, files.filename, block_hashes.file_offset, \
                block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize \
                FROM files \
                INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id;", image_conn)
print("original image length ",len(image_df))
image_df = image_df[image_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
image_df = image_df[image_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
image_df = image_df[image_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
image_df = image_df[image_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
print("current image length ",len(image_df))


    


original image length  4988815
current image length  586972


In [84]:
app_list = catalog_df.app.unique()
for app in app_list:
    #if app == 'OfficePro2003-W7x32':
    app_df = catalog_df[catalog_df.app == app]
    app_unique_md5s= app_df.md5.unique()
    matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
    #print(app," matched number of sectors in image ",len(matched_image_df))
    #print(matched_image_df)
    #lets create app sec pairs
    app_files = app_df.filename.unique()
    #print(app_files)
    app_pairs_set = set() 
    for file in app_files:
        files_df = app_df[app_df.filename == file]
        #print(f'file {file} is of size {len(files_df)}')
        file_hashes = files_df.md5
        if len(file_hashes)< 2:
            #print(file_hashes.iloc[0])
            app_pairs_set.add(file_hashes.iloc[0])
        else:
            for i in range(0, len(file_hashes)-1):
                #print(file_hashes.iloc[i])
                #print(file_hashes.iloc[int(i+1)])
                hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
            #print(i, hash_pair)
                app_pairs_set.add(hash_pair)
    #print(f"set size is {len(app_pairs_set)}")
    lst2 = list(matched_image_df.md5)
    lst2_pairs = list(map(lambda a, b: a + b, lst2[:-1], lst2[1:]))
    forward_list = list(map(lambda x: 1 if x in app_pairs_set else 0, lst2_pairs))
    forward_series = pd.Series(forward_list)
    print(app, "matched", len(matched_image_df), "set", len(app_pairs_set), "valid",forward_series.value_counts().to_dict())
        

Wireshark-W7x64 matched 56416 set 207316 valid {1: 55729, 0: 686}
Wireshark-W7x32 matched 171330 set 170020 valid {1: 170615, 0: 714}
Winzip17pro-W7x32 matched 184 set 244654 valid {0: 95, 1: 88}
Winzip17pro-W7x64 matched 95 set 267364 valid {0: 48, 1: 46}
sdelete-W7x32 matched 8 set 322 valid {0: 7}


/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_870/961406072.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  forward_series = pd.Series(forward_list)


sdelete-W7x64 matched 0 set 305 valid {}
OfficePro2003-WinXP matched 100 set 1180648 valid {0: 55, 1: 44}
OfficePro2003-W7x64 matched 143 set 626361 valid {0: 98, 1: 44}
OfficePro2003-W7x32 matched 374 set 594938 valid {1: 205, 0: 168}
Winrar5beta-W7x32 matched 39 set 9005 valid {0: 32, 1: 6}
Firefox19-WinXP matched 208 set 89882 valid {1: 170, 0: 37}
Winrar5beta-W7x64 matched 39 set 9027 valid {0: 32, 1: 6}
Firefox19-W7x32 matched 281 set 89927 valid {1: 170, 0: 110}
HxD171-W7x32 matched 27 set 4721 valid {0: 26}
Firefox19-W7x64 matched 239 set 96105 valid {1: 170, 0: 68}
Thunderbird2-WinXP matched 16 set 50877 valid {1: 11, 0: 4}
Python264-WinXP matched 19 set 82470 valid {1: 11, 0: 7}
eraser-W7x32 matched 51 set 5129 valid {0: 50}
Chrome28-W7x64 matched 207 set 594382 valid {1: 110, 0: 96}
Chrome28-WinXP matched 80 set 271911 valid {0: 41, 1: 38}
Chrome28-W7x32 matched 283 set 593981 valid {0: 170, 1: 112}
Safari157-W7x32 matched 236 set 295029 valid {0: 145, 1: 90}
Safari157-WinXP 

In [85]:
app_list = catalog_df.app.unique()
for app in app_list:
    if app == 'Firefox19-W7x64':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_dict_set = {}
        app_pairs_set = set() 
        for file in app_files:
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            if len(file_hashes)< 2:
                #print(file_hashes.iloc[0])
                #app_pairs_set.add(file_hashes.iloc[0])
                app_dict_set[file]=set(file_hashes.iloc[0])
            else:
                app_dict_set[file] = set()
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                    #print(i, hash_pair)
                    #app_pairs_set.add(hash_pair)
                    app_dict_set[file].add(hash_pair)
        #print(f"set size is {len(app_pairs_set)}")
        #for key,value in app_dict_set.items():
            #print(key,len(value))
        image_md5_list = list(matched_image_df.md5)
        image_md5_pairs = list(map(lambda a, b: a + b, image_md5_list[:-1], image_md5_list[1:]))
        for key,value in app_dict_set.items():
            forward_list = list(map(lambda x: 1 if x in value else 0, image_md5_pairs))
            #forward_x = list(map(lambda x: x if x in value else 0, image_md5_pairs))
            forward_series = pd.Series(forward_list).value_counts()
            if 1 in forward_series.index.values:
                print(key,forward_series[1])#,"\n",forward_x)
        #print(app, "matched", len(matched_image_df), "set", len(app_pairs_set), "valid",forward_series.value_counts())

Firefox19-W7x64  matched number of sectors in image  239
Program Files (x86)/Mozilla Firefox/d3dx9_43.dll 12
Program Files (x86)/Mozilla Firefox/D3DCompiler_43.dll 12
Program Files (x86)/Mozilla Firefox/msvcp100.dll 53
Program Files (x86)/Mozilla Firefox/mozjs.dll 1
Program Files (x86)/Mozilla Firefox/xul.dll 1
Program Files (x86)/Mozilla Firefox/webapprt-stub.exe 3
Program Files (x86)/Mozilla Firefox/msvcr100.dll 122


In [87]:
app_list = catalog_df.app.unique()
for app in app_list:
    if app == 'Firefox19-W7x64':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_dict_set = {}
        app_pairs_set = set() 
        for file in app_files:
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            if len(file_hashes)< 2:
                #print(file_hashes.iloc[0])
                #app_pairs_set.add(file_hashes.iloc[0])
                app_dict_set[file]=set(file_hashes.iloc[0])
            else:
                app_dict_set[file] = set()
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                    #print(i, hash_pair)
                    #app_pairs_set.add(hash_pair)
                    app_dict_set[file].add(hash_pair)
        #print(f"set size is {len(app_pairs_set)}")
        #for key,value in app_dict_set.items():
            #print(key,len(value))
        image_md5_list = list(matched_image_df.md5)
        image_md5_pairs = list(map(lambda a, b: a + b, image_md5_list[:-1], image_md5_list[1:]))
        for key,value in app_dict_set.items():
            forward_list = list(map(lambda x: 1 if x in value else 0, image_md5_pairs))
            forward_x = list(map(lambda x: x if x in value else 0, image_md5_pairs))
            forward_series = pd.Series(forward_list).value_counts()
            if 1 in forward_series.index.values:
                print(key,forward_series[1],"\n",forward_x)
        #print(app, "matched", len(matched_image_df), "set", len(app_pairs_set), "valid",forward_series.value_counts())
        

Firefox19-W7x64  matched number of sectors in image  239
Program Files (x86)/Mozilla Firefox/d3dx9_43.dll 12 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '1696b3148f687253c74c7f22f01b9e63698d3b7b4c51f4855cde5a51a028ebae', '698d3b7b4c51f4855cde5a51a028ebae4f014f8315537882802c8c083709a15f', '4f014f8315537882802c8c083709a15f70e1dc20d3d90245b56f5b3f16db24e3', '70e1dc20d3d90245b56f5b3f16db24e3d52b060ef3c295051db679b1c2a643a6', 'd52b060ef3c295051db679b1c2a643a64d8a5c39b2fdcd302c638b3b7fca35f5', '4d8a5c39b2fdcd302c638b3b7fca35f5fc4b1942ea87cf2a277a1055843bcf3f', 'fc4b1942ea87cf2a277a1055843bcf3f5d731cde0a84bb468eb6633c185b3ee6', 0, 0, 0, 0, 0